In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codebert_embeddings/df_dc_embeded_by_line_codebert_base.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_35293/1492473580.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,8077,critical,public class DruidPooledCallableStatement exte...,1,"[[-142.3737335205078, 271.9561767578125, -16.3...","[[-0.15341997146606445, 0.2930562198162079, -0..."
1,5553,critical,public interface IAopReferenceModel {\n\t\n\tv...,1,"[[-2.4264557361602783, 5.172459602355957, 0.18...","[[-0.13480310142040253, 0.2873588800430298, 0...."
2,9341,critical,@XmlAccessorType(XmlAccessType.FIELD)\n@XmlTyp...,1,"[[-5.216373920440674, 12.301513671875, -0.1164...","[[-0.14098307490348816, 0.3324733376502991, -0..."
3,10419,critical,public class Ipam {\n private String driver;\...,1,"[[-10.633790969848633, 23.82303237915039, -1.5...","[[-0.15637928247451782, 0.3503386974334717, -0..."
4,12232,critical,public class DruidPooledCallableStatement exte...,1,"[[-142.3737335205078, 271.9561767578125, -16.3...","[[-0.15341997146606445, 0.2930562198162079, -0..."


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,8077,critical,1,"[[-142.3737335205078, 271.9561767578125, -16.3..."
1,5553,critical,1,"[[-2.4264557361602783, 5.172459602355957, 0.18..."
2,9341,critical,1,"[[-5.216373920440674, 12.301513671875, -0.1164..."
3,10419,critical,1,"[[-10.633790969848633, 23.82303237915039, -1.5..."
4,12232,critical,1,"[[-142.3737335205078, 271.9561767578125, -16.3..."


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[-142.3737335205078, 271.9561767578125, -16.3...
1    [[-2.4264557361602783, 5.172459602355957, 0.18...
2    [[-5.216373920440674, 12.301513671875, -0.1164...
3    [[-10.633790969848633, 23.82303237915039, -1.5...
4    [[-142.3737335205078, 271.9561767578125, -16.3...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3694
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[-142.3737335205078, 271.9561767578125, -16.3...
1    [[-2.4264557361602783, 5.172459602355957, 0.18...
2    [[-5.216373920440674, 12.301513671875, -0.1164...
3    [[-10.633790969848633, 23.82303237915039, -1.5...
4    [[-142.3737335205078, 271.9561767578125, -16.3...
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 3670


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [-142.3737335205078, 271.9561767578125, -16.33...
1    [-2.4264557361602783, 5.172459602355957, 0.183...
2    [-5.216373920440674, 12.301513671875, -0.11644...
3    [-10.633790969848633, 23.82303237915039, -1.59...
4    [-142.3737335205078, 271.9561767578125, -16.33...
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3694
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


        emb_1       emb_2      emb_3      emb_4      emb_5       emb_6  \
0 -142.373734  271.956177 -16.337118  51.671291  76.822624 -245.059525   
1   -2.426456    5.172460   0.183453   1.040403   0.761608   -3.009145   
2   -5.216374   12.301514  -0.116449   2.123092   3.188100   -7.073945   
3  -10.633791   23.823032  -1.599639   3.053346   3.419122  -15.894526   
4 -142.373734  271.956177 -16.337118  51.671291  76.822624 -245.059525   

       emb_7      emb_8       emb_9      emb_10  ...     emb_762    emb_763  \
0  27.996546  38.375412  261.649292 -244.260834  ...  512.339355 -60.587494   
1  -0.883193   0.836070    2.778491   -2.877727  ...   10.506008  -3.874928   
2  -0.892327   1.628437    7.502768   -8.853976  ...   19.812071  -4.468370   
3  -2.235992   3.425864   15.858608  -15.242412  ...   30.434586  -7.688050   
4  27.996546  38.375412  261.649292 -244.260834  ...  512.339355 -60.587494   

     emb_764     emb_765     emb_766     emb_767     emb_768  label  \
0 -10.584

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[-0.15341997146606445, 0.2930562198162079, -0...
1    [[-0.13480310142040253, 0.2873588800430298, 0....
2    [[-0.14098307490348816, 0.3324733376502991, -0...
3    [[-0.15637928247451782, 0.3503386974334717, -0...
4    [[-0.15341997146606445, 0.2930562198162079, -0...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3694
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 16294


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 2754


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [-0.15341997146606445, 0.2930562198162079, -0....
1    [-0.13480310142040253, 0.2873588800430298, 0.0...
2    [-0.14098307490348816, 0.3324733376502991, -0....
3    [-0.15637928247451782, 0.3503386974334717, -0....
4    [-0.15341997146606445, 0.2930562198162079, -0....
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3694
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.153420,0.293056,-0.017605,0.055680,0.082783,-0.264073,0.030169,0.041353,0.281950,-0.263212,...,0.552090,-0.065288,-0.011406,0.399021,-0.295382,-0.346146,0.336526,1,8077,critical
1,-0.134803,0.287359,0.010192,0.057800,0.042312,-0.167175,-0.049066,0.046448,0.154361,-0.159874,...,0.583667,-0.215274,0.060102,0.400974,-0.245126,-0.341638,0.333059,1,5553,critical
2,-0.140983,0.332473,-0.003147,0.057381,0.086165,-0.191188,-0.024117,0.044012,0.202778,-0.239297,...,-0.458214,0.535461,-0.120767,0.027277,0.383095,-0.258017,-0.341915,1,9341,critical
3,-0.156379,0.350339,-0.023524,0.044902,0.050281,-0.233743,-0.032882,0.050380,0.233215,-0.224153,...,0.447567,-0.113060,0.024070,0.371381,-0.312726,-0.301403,0.337536,1,10419,critical
4,-0.153420,0.293056,-0.017605,0.055680,0.082783,-0.264073,0.030169,0.041353,0.281950,-0.263212,...,0.552090,-0.065288,-0.011406,0.399021,-0.295382,-0.346146,0.336526,1,12232,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)